### PYABSA INDOBERT + NER

In [2]:
!pip install transformers torch datasets spacy -q
!python -m spacy download xx_ent_wiki_sm   # multilingual NER model

import pandas as pd
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 78.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('xx_ent_wiki_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
model_name = "cahya/bert-base-indonesian-NER"   # IndoBERT fine-tuned untuk NER
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of the model checkpoint at cahya/bert-base-indonesian-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, grouped_entities=True)

Device set to use cuda:0
/usr/local/lib/python3.12/dist-packages/transformers/pipelines/token_classification.py:186: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  warnings.warn(


In [6]:
df = pd.read_csv("/gabungan_dataset.csv")   # pastikan kolom Cleaned_Tweet ada

In [7]:
candidates = []
for text in df["Cleaned_Tweet"].dropna().tolist():
    entities = ner_pipeline(text)
    aspects = [ent["word"] for ent in entities] if entities else []
    candidates.append({"text": text, "candidate_aspects": aspects})

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [8]:
df_aspects = pd.DataFrame(candidates)
df_aspects.to_csv("calon_aspek.csv", index=False, encoding="utf-8-sig")

print("✅ Calon aspek disimpan di 'calon_aspek.csv'")
df_aspects.head()

✅ Calon aspek disimpan di 'calon_aspek.csv'


,text,candidate_aspects
0,kabinet merah putih mendukung kebijakan perlin...,"[kabinet, merah putih]"
1,istana pastikan pejabat kabinet merah putih pa...,"[kabinet, mobil dinas, pind]"
2,tidak akan pakai mobil toyota alphard tetapi p...,"[mobil toyota alphard, mobil mau, anggito, pra..."
3,mengikuti retreat di akmil magelang para mente...,"[##reat, akmil, magelang, indonesia, kabinet, ..."
4,retreat kabinet merah putih berakhir bawa pula...,"[kabinet, merah putih, jakarta]"


In [10]:
# Simpan ke Excel
df_aspects = pd.DataFrame(candidates)
df_aspects.to_excel("calon_aspek.xlsx", index=False)   # tanpa encoding

print("✅ Calon aspek disimpan di 'calon_aspek.xlsx'")
df_aspects.head()

✅ Calon aspek disimpan di 'calon_aspek.xlsx'


,text,candidate_aspects
0,kabinet merah putih mendukung kebijakan perlin...,"[kabinet, merah putih]"
1,istana pastikan pejabat kabinet merah putih pa...,"[kabinet, mobil dinas, pind]"
2,tidak akan pakai mobil toyota alphard tetapi p...,"[mobil toyota alphard, mobil mau, anggito, pra..."
3,mengikuti retreat di akmil magelang para mente...,"[##reat, akmil, magelang, indonesia, kabinet, ..."
4,retreat kabinet merah putih berakhir bawa pula...,"[kabinet, merah putih, jakarta]"


### PYABSA INDOBERT